# 01 Práce s databází pomocí ADO.NET

**autor: Erik Král ekral@utb.cz**

---

V tomto materiálu si probereme práci s databází pomocí knihovny ADO.NET.

Nejprve si nainstalujeme nuget balíček [Microsoft.Data.Sqlite](https://www.nuget.org/packages/Microsoft.Data.SQLite/) představující database providera pro souborovou databázi *Sqlite*. Tento balíček používají následující příklady.

In [ ]:
#r "nuget: Microsoft.Data.Sqlite"

## Připojení k databázi

Dále si vytvoříme connection string tak, aby se soubor s databází ukládal do dokumentů uživatele. 

In [ ]:
using Microsoft.Data.Sqlite;
using System.IO;

var folder = Environment.SpecialFolder.MyDocuments;
string folderPath = Environment.GetFolderPath(folder);
string filePath = Path.Join(folderPath, "skola.db");

SqliteConnectionStringBuilder csb = new SqliteConnectionStringBuilder
{
    DataSource = filePath
};

string connectionString = csb.ConnectionString;

Console.WriteLine(connectionString);

K databázi se připojíme pomocí následujícího příkazu. Pokud Sqlite databáze ještě neexistuje, tak se vytvoří nová (nový soubor). Třída *SqliteConnection* implemetuje rozhraní *IAsyncDisposable* a proto se nám připojení uzavře automaticky s využití Dispose patternu a nemusíme jej už ručně uzavírat. 

In [ ]:
SqliteConnection connection;

await using(connection = new SqliteConnection(connectionString))
{
    await connection.OpenAsync();

    Console.WriteLine(connection.State);
}

Console.WriteLine(connection.State);

## Zadávání dat pomocí ExecuteNonQueryAsync

Pro **vytvoření tabulky**, **vložení nového řádku**, nebo **aktualizaci řádku**, tedy příkazy, které neprovádějí dotaz a nevrací hodnoty používáme metodu ```command.ExecuteNonQuery()```. 

### Vytvoření tabulky

V následujícím kódu vytvoříme novou tabulku `Studenti`, která má sloupce `StudentId`, `Jmeno` a dosažené `Kredity`:

In [ ]:
await using(SqliteConnection connection = new SqliteConnection(connectionString))
{
    await connection.OpenAsync();

    await using(SqliteCommand command = connection.CreateCommand())
    {
        command.CommandText =
        @"
            CREATE TABLE Studenti 
            (
                StudentId INTEGER PRIMARY KEY, 
                Jmeno TEXT,
                Kredity INTEGER
            )
        ";

        int count = await command.ExecuteNonQueryAsync();

        Console.WriteLine($"Počet změněných řádků: {count}");
    }
}

### Vložení nového řádku

Nyní do vytvořené tabulky vložíme nový řádek. V příkladu používáme parametry commandu `@Jmeno` a `@Kredity`. Hodnoty parametrů můžeme zadávat buď s generickými SQL typy. Nebo s konkrétními typy pro danou databází s metodou, kterou poskytuje konkrétní provider tak jak je ukázáno v příkladu. 

Použití parametrů proskytuje kontrolu typu a validaci zadané hodnoty parametru a pomáhá zabránit útoku technikou SQL Injection. Použití parametrů nezpomaluje provedení dotazu, spíše nám může pomoct dotaz lépe sestavit dle konkrétního typu a díky tomu by provedení dotazu by mohlo být v některých případech efektivnější.

In [ ]:
await using(SqliteConnection connection = new SqliteConnection(connectionString))
{
    await connection.OpenAsync();

    await using(SqliteCommand command = connection.CreateCommand())
    {
        command.CommandText =
        @"
            INSERT INTO Studenti (Jmeno, Kredity)
            VALUES (@Jmeno, @Kredity)
        ";

        command.Parameters.Add("@Jmeno", SqliteType.Text).Value = "Karel";
        command.Parameters.Add("@Kredity", SqliteType.Integer).Value = 40;

        int count = await command.ExecuteNonQueryAsync();

        Console.WriteLine($"Počet změněných řádků: {count}");
    }
}

### Změna řádku

Hodnoty řádku změníme obdobým způsobem jako přidávání nových. Konkrétně změníme počet kreditů studenta se `StudentId` 1.

In [ ]:
await using(SqliteConnection connection = new SqliteConnection(connectionString))
{
    await connection.OpenAsync();

    await using(SqliteCommand command = connection.CreateCommand())
    {
        command.CommandText =
        @"
            UPDATE Studenti
            SET Kredity = @Kredity 
            WHERE StudentId = @StudentId
        ";

        command.Parameters.Add("@Kredity", SqliteType.Integer).Value = 64;
        command.Parameters.Add("@StudentId", SqliteType.Integer).Value = 1;

        int count = await command.ExecuteNonQueryAsync();

        Console.WriteLine($"Počet změněných řádků: {count}");
    }
}

## Načtení dat pomocí ExecuteReaderAsync

Pokud chceme provést dotaz na data tabulky a načíst jednotlivé řádky a sloupce, tak použijeme `reader` jak je uvedeno v následujícím kódu. 

In [ ]:
await using(SqliteConnection connection = new SqliteConnection(connectionString))
{
    await connection.OpenAsync();

    await using(SqliteCommand command = connection.CreateCommand())
    {
        command.CommandText = "SELECT StudentId, Jmeno, Kredity FROM Studenti";

        SqliteDataReader reader = await command.ExecuteReaderAsync();

        if(reader.HasRows)
        {
            while(await reader.ReadAsync())
            {
            
                int studentId = reader.GetInt32(reader.GetOrdinal("StudentId"));
                string jmeno = reader.GetString(reader.GetOrdinal("Jmeno"));
                int kredity = reader.GetInt32(reader.GetOrdinal("Kredity"));

                Console.WriteLine($"{studentId}: {jmeno} {kredity}");
            }
        }
    }
}

## Načtení jedné hodnoty pomocí ExecuteScalar

Prokud provedeme SQL dotaz, který vrátí pouze jednu hodnotu, například ```SELECT AVG(Kredit) FROM Studenti```, tak můžeme použít ```reader``` podobně jako v minulém příkladu a načíst první sloupec prvního řádku. A nebo můžeme použít metodu ```command.ExecuteScalar()``` tak jak je ukázané v následujícím příkladu.

In [ ]:
await using(SqliteConnection connection = new SqliteConnection(connectionString))
{
    await connection.OpenAsync();

    await using(SqliteCommand command = connection.CreateCommand())
    {
        command.CommandText =@$"SELECT AVG(Kredity) FROM Studenti";

        object? result = await command.ExecuteScalarAsync();

        if(result is double average)
        {
            return average;
        }
    }
}

## Relace a transakce

Nyní si vytvoříme příklad na relace a transakce. Nejprve vytvoříme tabulku `Majitel` a `Ucet` reprezentující bankovní účet a jeho majitele. Účet bude cizí klíč `MajitelId`. Vazba je tedy 1:n kdy účet může mít jen jednoho majitele, ale majitel může mít více účtů. Vlastní relaci definuje řádek `FOREIGN KEY (MajitelId) REFERENCES Majitel(MajitelId)`. V příkladu používáme také [transakce](https://learn.microsoft.com/cs-cz/dotnet/framework/data/adonet/local-transactions), kdy transakci potvrdíme, teprve až se nám podaří vytvořit obě tabulky.

In [ ]:
await using(SqliteConnection connection = new SqliteConnection(connectionString))
{
    await connection.OpenAsync();

    SqliteTransaction transaction = connection.BeginTransaction();

    await using(SqliteCommand command = connection.CreateCommand())
    {
        command.Transaction = transaction;

        try
        {
            command.CommandText =
            @"
                CREATE TABLE Majitel 
                (
                    MajitelId INTEGER PRIMARY KEY, 
                    Jmeno TEXT
                )
            ";

            await command.ExecuteNonQueryAsync();

            command.CommandText =
            @"
                CREATE TABLE Ucet
                (
                    UcetId INTEGER PRIMARY KEY, 
                    Nazev TEXT,
                    Zustatek REAL,
                    MajitelId INTEGER,
                    FOREIGN KEY (MajitelId) REFERENCES Majitel(MajitelId)
                )
            ";

            await command.ExecuteNonQueryAsync();

            await transaction.CommitAsync();
        }
        catch (Exception ex)
        {
            Console.WriteLine(ex.Message);
            
            await transaction.RollbackAsync();
        }

    }
}

Nyní si nadefinujeme a zavoláme metodu, která přidá nové majitele. Všimněte si příkazu `RETURNING`, což je specifický příkaz SQL dialektu Sqlite, který vrátí hodnotu vygenerovaného primární klíče.

In [ ]:
async Task AddMajitel(string jmeno)
{
    await using(SqliteConnection connection = new SqliteConnection(connectionString))
    {
        await connection.OpenAsync();


        await using(SqliteCommand command = connection.CreateCommand())
        {
            command.CommandText =
            @"
                INSERT INTO Majitel (Jmeno) 
                VALUES (@Jmeno) 
                RETURNING MajitelId
            ";

            command.Parameters.Add("@Jmeno", SqliteType.Integer).Value = jmeno;

            object? result = await command.ExecuteScalarAsync();

            if(result is Int64 majitelId)
            {
                Console.WriteLine($"MajitelId: {majitelId}");
            }
            
        }
    }
}

await AddMajitel("Jiri");
await AddMajitel("Karel");

Dále si nadefinujeme a zavoláme metodu `AddUcet`, kdy majiteli s `MajitelId` `1` přiřadíme spořící účet a majiteli s `MajitelId` `2` běžný účet.

In [ ]:
async Task AddUcet(string nazev, double zustatek, int majitelId)
{
    await using(SqliteConnection connection = new SqliteConnection(connectionString))
    {
        await connection.OpenAsync();

  
        await using(SqliteCommand command = connection.CreateCommand())
        {
            command.CommandText =
            @"
                INSERT INTO Ucet (Nazev, Zustatek, MajitelId) 
                VALUES (@Nazev, @Zustatek, @MajitelId) 
                RETURNING UcetId
            ";

            command.Parameters.Add("@Nazev", SqliteType.Integer).Value = nazev;
            command.Parameters.Add("@Zustatek", SqliteType.Real).Value = zustatek;
            command.Parameters.Add("@MajitelId", SqliteType.Integer).Value = majitelId;

            object? result = await command.ExecuteScalarAsync();

            if(result is Int64 ucetId)
            {
                Console.WriteLine($"UcetId: {ucetId}");
            }
            
        }
    }
}

await AddUcet("sporici", 1000.0, 1);
await AddUcet("bezny", 3000.0, 2);

Další příklad představuje využití SQL příkazu `JOIN` kdy vrátíme uživatele a jejich účty.

In [ ]:
await using(SqliteConnection connection = new SqliteConnection(connectionString))
{
    await connection.OpenAsync();

    await using(SqliteCommand command = connection.CreateCommand())
    {
        command.CommandText = 
        @"
            SELECT Majitel.Jmeno, Ucet.Nazev, Ucet.Zustatek
            FROM Majitel
            INNER JOIN Ucet ON Majitel.MajitelId=Ucet.MajitelId
        ";

        SqliteDataReader reader = await command.ExecuteReaderAsync();

        if(reader.HasRows)
        {
            while(await reader.ReadAsync())
            {
                string jmeno = reader.GetString(0);
                string nazev = reader.GetString(1);
                double zustatek = reader.GetDouble(2);

                Console.WriteLine($"{jmeno} {nazev} {zustatek}");
            }
        }
    }
}

Poslední příklad představuje další ukázku transakce. Konkrétně chceme převést částku z jednohu účtu na druhý. V tomto případě tedy od jednoho účtu částku odečteme a k druhému přičteme. Pokud bych nepoužili transakce, tak by se nám při selhání příkazu mohlo stát, že od účtu částku odečteme ale k druhému ji nepřičteme a v databázi bychom měli nesprávná data.

In [ ]:
async Task Vyber(int ucetId1, int ucetId2, double castka)
{
    await using(SqliteConnection connection = new SqliteConnection(connectionString))
    {
        await connection.OpenAsync();

        SqliteTransaction transaction = connection.BeginTransaction();

        await using(SqliteCommand command = connection.CreateCommand())
        {
            command.Transaction = transaction;

            command.Parameters.Add("@Castka", SqliteType.Real).Value = castka;
            command.Parameters.Add("@UcetId", SqliteType.Integer);

            try
            {
                command.CommandText =
                @"
                    UPDATE Ucet 
                    SET Zustatek = Zustatek - @Castka 
                    WHERE UcetId = @UcetId;
                ";

                command.Parameters["@UcetId"].Value = ucetId1;

                await command.ExecuteNonQueryAsync();

                command.CommandText =
                @"
                    UPDATE Ucet 
                    SET Zustatek = Zustatek + @Castka 
                    WHERE UcetId = @UcetId;
                ";

                command.Parameters["@UcetId"].Value = ucetId2;
                
                await command.ExecuteNonQueryAsync();

                await transaction.CommitAsync();
            }
            catch (Exception ex)
            {
                Console.WriteLine(ex.Message);

                await transaction.RollbackAsync();
            }

        }
    }
}

await Vyber(1, 2, 100.0);